In [ ]:
using Pkg; Pkg.activate(".")
using SparseArrays, LinearAlgebra, Ferrite
using FerriteViz, WGLMakie, Makie, LaTeXStrings
Makie.inline!(true);

include("ferrite_tools.jl")
include("aitken.jl") 

In this notebook we will use the code `Ferrite.jl` to solve our simplest model problem, 
$$\begin{aligned} 
 - \Delta u  &= f, \qquad \Omega, \\ 
          u &= 0, \qquad \partial\Omega
\end{aligned}$$ 
In the assignment we will generalize this to some more intersting problems.

In [ ]:
grid, ∂Ω = generate_square(20)

FerriteViz.wireframe(grid, markersize = 10, strokewidth = 2, 
                     figure = (resolution=(500,500),))

In [ ]:
# solve the dirichlet problem
ffun = ξ -> 5.0
k = 1
cellvalues, dh, ch, K = setup_fem(k, grid, ∂Ω)
u = solve_fem(cellvalues, dh, ch, K, ffun);
# visualize the solution
plotter = FerriteViz.MakiePlotter(dh, u)
fig = FerriteViz.surface(plotter, field=:u, 
                   figure = (resolution = (700,700,),))

We don't know the exact solution in this case. Here is a little trick 
how we can still look at the error. Instead of plotting the convergence of $\|u - u_h \|_{H^1}$ we plot the convergence of $|J(u) - J(u_h)| = \frac12 \| u - u_h \|_{H^1}^2$. To get the reference energy we can extrapolate from the sequence of computed energies. This is a bit dicey with higher-order FEM because the errors get so small. We therefore restrict our tests to P1 and P2.

In [ ]:
function compute_energy(k, N, generate_dom, ffun)
    grid, ∂Ω = generate_dom(N)
    cellvalues, dh, ch, K = setup_fem(k, grid, ∂Ω)
    K, f = assemble_global!(cellvalues, dh, K, ξ -> ffun(ξ));
    apply!(K, f, ch)
    u = K \ f;
    return 0.5 * dot(u, K * u) - dot(u, f)
end

In [ ]:

ffun = ξ -> 5.0
NN = [4, 8, 16, 32, 64]
E = [ [ compute_energy(k, N, generate_square, ffun) for N in NN ]
      for k = 1:2 ]
Elim = aitken.(E)
err = [ abs.(E[i] .- Elim[i]) for i = 1:2 ]
;

In [ ]:
# show the convergence of total energy 
fig = Figure(size = (400, 400); fontsize=30)
ax = Axis(fig[1, 1], xlabel = L"h^{-1}", ylabel = L"|E(u) - E(u_h)|", 
          xscale = log10, yscale = log10,
          title = "Error P1-FEM")
NN1 = NN[3:5]
scatterlines!(NN, err[1]; linewidth=5, markersize=20, label=L"P1")
scatterlines!(NN, err[2]; linewidth=5, markersize=20, label=L"P2")
lines!(NN1, 5 ./ NN1.^(2); color=:black, linewidth=3, label=L"h^2, h^4") 
lines!(NN1, 5 ./ NN1.^(4); color=:black, linewidth=3) 

axislegend(ax)
fig

Now let's do the same on a more interesting domain, the L-shape $\Omega = (-1, 1)^2 \setminus (0, 1)^2$ (actually we are removing a different square but it doesn't really matter.

In [ ]:
grid, ∂Ω = generate_lshape(20)

FerriteViz.wireframe(grid, markersize = 10, strokewidth = 2, 
                     figure = (resolution=(500,500),))

In [ ]:
# we can assemble the system
ffun = ξ -> 10.0
k = 1
cellvalues, dh, ch, K = setup_fem(k, grid, ∂Ω)
u = solve_fem(cellvalues, dh, ch, K, ffun);
# visualize the solution
plotter = FerriteViz.MakiePlotter(dh, u)
fig = FerriteViz.surface(plotter, field=:u, 
                   figure = (resolution = (700,700,),))

In [ ]:

ffun = ξ -> 10.0
NN = [4, 8, 16, 32, 64, 128, 256]
E = [ [ compute_energy(k, N, generate_lshape, ffun) for N in NN ]
      for k = 1:2 ]
Elim = aitken.(E)
err = [ abs.(E[i] .- Elim[i]) for i = 1:2 ];


In [ ]:
# show the convergence of total energy 
fig = Figure(size = (400, 400); fontsize=30)
ax = Axis(fig[1, 1], xlabel = L"h^{-1}", ylabel = L"|E(u) - E(u_h)|", 
          xscale = log10, yscale = log10,
          title = "Error P1-FEM")
NN1 = NN[3:5]
scatterlines!(NN, err[1]; linewidth=5, markersize=20, label=L"P1")
scatterlines!(NN, err[2]; linewidth=5, markersize=20, label=L"P2")
lines!(NN1, 5 ./ NN1.^(2); color=:black, linewidth=3, label=L"h^2, h^4") 
lines!(NN1, 5 ./ NN1.^(4); color=:black, linewidth=3) 

axislegend(ax)
fig

Oh dear, our predicted rates are completely off! With a bit of experimenting we observe a rate of $h^{4/3}$ for the energy which corresponds to $h^{2/3}$ for the solution. 

In [ ]:
# show the convergence of total energy 
fig = Figure(size = (400, 400); fontsize=30)
ax = Axis(fig[1, 1], xlabel = L"h^{-1}", ylabel = L"|E(u) - E(u_h)|", 
          xscale = log10, yscale = log10,
          title = "Error P1 and P2-FEM")
NN1 = NN[4:7]
scatterlines!(NN, err[1]; linewidth=5, markersize=20, label=L"P1")
scatterlines!(NN, err[2]; linewidth=5, markersize=20, label=L"P2")
lines!(NN1, 2 ./ NN1.^(4/3); color=:black, linewidth=3, label=L"h^{4/3}") 

axislegend(ax)
fig

## Method of Manufactured Solutions

We can produce another example so clearly see the $h^{2/3}$ convergence rate for the solution as well, using the method of manufactured solutions. 
As before, we specify a solution $u(x)$ and compute $f = - \Delta u(x)$ via automatic differentiation. Then we use $f$ as the input into our FEM.

But we take care that we give the solution the natural leading singularity. We do it for $N = 2$ where we can see the predicted rate more clearly.

In [ ]:
using ForwardDiff, LinearAlgebra

function u_ex(x)
    r = norm(x) 
    if r == 0.0; return 0.0; end 
    θ = mod(atan(x[2], x[1]), 2*π)
    return 3 * (1 - x[1]^2) * (1 - x[2]^2) * r^(2/3) * sin(2/3*θ)
end

;

This might feel a bit fake, but the resulting forcing function is actually smooth, in particular bounded, which we can quickly confirm: 

In [ ]:
ffun = x -> - tr( ForwardDiff.hessian(u_ex, x) )
grid, _ = generate_lshape(40)
maxf_nodes = maximum( abs(ffun(2*rand(2) .- 1)) for _ = 1:10_000)

Equipped with some confidence in our model problem we can study the convergence:

In [ ]:
NN = [4, 8, 16, 32, 64, 128]
errs_L2 = Float64[] 
errs_H1 = Float64[]
for N in NN
    err_L2, err_H1 = fem_errors(2, N, generate_lshape, u_ex)
    push!(errs_L2, err_L2)
    push!(errs_H1, err_H1)
end
;

In [ ]:
# plot the errors
fig = Figure(size = (400, 400); fontsize=30)
ax = Axis(fig[1, 1], xlabel = L"h^{-1}", ylabel = L"\text{error}", 
          xscale = log10, yscale = log10,
          title = "Error P2-FEM")
scatterlines!(NN, errs_L2; linewidth=5, markersize=20, label=L"L^2")
scatterlines!(NN, errs_H1; linewidth=5, markersize=20, label = L"H^1")
NN1 = NN[3:5]
lines!(NN1, 1.5 ./ NN1.^(2/3); color=:black, linewidth=3, label = L"h^{2/3}, h^{4/3}")
lines!(NN1, 0.3 ./ NN1.^(4/3); color=:black, linewidth=3)
axislegend(ax)
fig

## Final Example

Consider a domain with a wedge cut out as shown in the next figure. 

In [ ]:
grid, ∂Ω = generate_wedge(20)

FerriteViz.wireframe(grid, markersize = 10, strokewidth = 2, 
                     figure = (resolution=(500,500),))

What will the rate of convergence be in this case -- according to our theory? 

We will study this in energy to make sure we don't pre-assume anything accidentlly.

In [ ]:

ffun = ξ -> 10.0
NN = [4, 8, 16, 32, 64, 128, 256]
E = [ [ compute_energy(k, N, generate_wedge, ffun) for N in NN ]
      for k = 1:2 ]
Elim = aitken.(E)
err = [ abs.(E[i] .- Elim[i]) for i = 1:2 ];


In [ ]:
# show the convergence of total energy : 7/4 π -> ???
fig = Figure(size = (400, 400); fontsize=30)
ax = Axis(fig[1, 1], xlabel = L"h^{-1}", ylabel = L"|E(u) - E(u_h)|", 
          xscale = log10, yscale = log10,
          title = "Error P1-FEM")
NN1 = NN[4:7]
scatterlines!(NN, err[1]; linewidth=5, markersize=20, label=L"P1")
scatterlines!(NN, err[2]; linewidth=5, markersize=20, label=L"P2")

# fill this in!
# lines!(NN1, 2.5 ./ NN1.^(***); color=:black, linewidth=3, label=L"h^{4/3}") 

axislegend(ax)
fig